In [1]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def nlogn_median(l):
    l = sorted(l)
    if len(l) % 2 == 1:
        return l[len(l) // 2]
    else:
        return 0.5 * (l[len(l) // 2 - 1] + l[len(l) // 2])


In [3]:
def quickselect(l, k, pivot_fn):
    """
    Выбираем k-тый элемент в списке l (с нулевой базой)
    :param l: список числовых данных
    :param k: индекс
    :param pivot_fn: функция выбора pivot, по умолчанию выбирает случайно
    :return: k-тый элемент l
    """
    if len(l) == 1:
#         assert k == 0
        return l[0]

    pivot = pivot_fn(l)

    lows = [el for el in l if el < pivot]
    highs = [el for el in l if el > pivot]
    pivots = [el for el in l if el == pivot]

    if k < len(lows):
        return quickselect(lows, k, pivot_fn)
    elif k < len(lows) + len(pivots):
        # Нам повезло и мы угадали медиану
        return pivots[0]
    else:
        return quickselect(highs, k - len(lows) - len(pivots), pivot_fn)

In [4]:
def quickselect_median(l, pivot_fn=np.random.choice):
    if len(l) % 2 == 1:
        return quickselect(l, len(l) / 2, pivot_fn)
    else:
        return 0.5 * (quickselect(l, len(l) / 2 - 1, pivot_fn) +
                      quickselect(l, len(l) / 2, pivot_fn))


In [5]:
def pick_pivot(l):
    """
    Выбираем хорошй pivot в списке чисел l
    Этот алгоритм выполняется за время O(n).
    """
    assert len(l) > 0

    # Если элементов < 5, просто возвращаем медиану
    if len(l) < 5:
        return nlogn_median(l)

    # Сначала разделим l на группы по 5 элементов. O(n)
    full_chunks = chunked(l, 5)

    # Для простоты мы можем отбросить все группы, которые не являются полными. O(n)
    # Медиана каждого фрагмента имеет индекс 2
    medians = [np.median(chunk) for chunk in full_chunks]
#     medians += np.median(full_chunks[-1])

    median_of_medians = quickselect_median(medians, pick_pivot)
    return median_of_medians

def chunked(l, chunk_size):
    """Разделяем список `l` на фрагменты размером `chunk_size`."""
    return [l[i:i + chunk_size] for i in range(0, len(l), chunk_size)]

In [6]:
ls = np.asarray([*range(0, 15, 3)])
pick_pivot(ls), np.median(ls)

(6.0, 6.0)

In [ ]:
df = pd.DataFrame({}, columns=['time', 'count'])

for i in range(10, int(1e6), 10000):
    tmp = np.random.randint(0, 10, i)
    start_time = time.time()
    med = pick_pivot(tmp)
    df.loc[i] = [i, time.time() - start_time]
    print((med, np.median(tmp), tmp) if med != np.median(tmp) else '')



(4.0, 5.0, array([1, 5, 8, ..., 9, 4, 5]))


(5.0, 4.0, array([8, 4, 6, ..., 6, 4, 6]))




(4.0, 5.0, array([6, 7, 3, ..., 8, 5, 6]))

(5.0, 4.0, array([3, 2, 1, ..., 9, 4, 6]))





In [ ]:
from scipy.stats import linregress
x, y = df['count'].to_numpy(), df['time'].to_numpy()
slope, intercept, r, p, se = linregress(x, y)
plt.plot(x, y, 'o', label='original data')
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.legend()
plt.show()

In [ ]:
np.median([1])